Este ejemplo esta basado en el tutorial de la documentación oficial de MLFlow: https://www.mlflow.org/docs/latest/tutorials-and-examples/tutorial.html.

Para ejecutar localmente esta notebook, necesitarás:
- Conda
- Scikit-learn
- MLFlow

In [ ]:
!pip install scikit-learn mlflow

Como ejemplo, vamos a entrenar un modelo de regresión lineal con scikit-learn. Para ello ejecutamos el script de python `train.py`, dentro de la carpeta del proyecto, `sklearn_elastic_wine`. Vamos a explorar y entendr este archivo, que utiliza MLFlow tracking para llevar un registro de los experimentos realizados.

Para ejecutar el entrenamiento podemos especificar los parámetros `alpha` y `l1_ratio` del modelo "elastic net" que estamos usando para regularizar la regresión lineal. Lo ejecutamos como

``` bash
python sklearn_elasticnet_wine/train.py <alpha> <l1_ratio>
```

Probemos algunos parámetros:

In [ ]:
!python sklearn_elasticnet_wine/train.py 0.6 0.7

Elasticnet model (alpha=0.600000, l1_ratio=0.700000):
  RMSE: 0.8321502650365332
  MAE: 0.6662000872414003
  R2: 0.018845859373919027


Ahora vamos a recordar como visualizar los modelos en la UI de mlflow, ejecutemos:

In [ ]:
!mlflow ui

[2021-11-22 16:41:00 -0600] [850163] [INFO] Starting gunicorn 20.1.0
[2021-11-22 16:41:00 -0600] [850163] [INFO] Listening at: http://127.0.0.1:5000 (850163)
[2021-11-22 16:41:00 -0600] [850163] [INFO] Using worker: sync
[2021-11-22 16:41:00 -0600] [850173] [INFO] Booting worker with pid: 850173
^C
[2021-11-22 16:42:28 -0600] [850163] [INFO] Handling signal: int
[2021-11-22 16:42:28 -0600] [850173] [INFO] Worker exiting (pid: 850173)


Ahora, digamos que nuestro modelo ya está listo. Queremos compartirlo, y para ello vamos a empaquetarlos usando MLFlow projects.
Projects funciona definiendo los puntos de entrada y las dependencias necesarias para ejecutar el modelo en un archivo especial de configuración llamado `MLproject`. Tenemos un ejemplo en la carpeta `sklearn_elasticnet_wine`, vamos a explorarlo.

Como podemos aprecias, el archivo de configuración nos indica que las dependencias están especificadas en otro archivo llamado `conda.yaml` y nuestro único entry point es el llamado main (como la función que lo ejecuta) que acepta dos parámetros: `alpha` y `l1_ratio`, ambos floats. Igualmente provee el comando correcto para invocar el entry point.

Ahora, si no nos dan estos archivos, ¿cómo generamos los propios? El archivo `conda.yaml` lo podemos obtener del run que prefiramos dentro de la carpeta mlruns, con algunos ajustes si es necesario. Este archivo es un artifact de cada run que se genera automaticamente. El archivo `MLproject` lo construimos nostros, con conocimiento de como funciona nuestro modelo. Podemos usar el ejemplo que proveemos aquí como base.

Para ejecutar el proyecto, usamos el comando

``` bash
mlflow run sklearn_elasticnet_wine -e <entry point> -P alpha=<alpha> -P l1_ratio=<l1_ratio>
```

In [ ]:
!mlflow run sklearn_elasticnet_wine -P alpha=0.42 -P l1_ratio=0.63

2021/11/22 17:02:59 INFO mlflow.utils.conda: === Creating conda environment mlflow-7122f0cb71f385d249fbb61cc599afd8045ab238 ===
Solving environment: done

python_abi-3.7       | 4 KB      | ##################################### | 100% 
setuptools-59.2.0    | 1.0 MB    | ##################################### | 100% 
python-3.7.12        | 57.3 MB   | ##################################### | 100% 
libnsl-2.0.0         | 31 KB     | ##################################### | 100% 
pip-21.3.1           | 1.2 MB    | ##################################### | 100% 
libffi-3.4.2         | 57 KB     | ##################################### | 100% 
openssl-3.0.0        | 2.9 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Installing pip dependencies: - Ran pip subprocess with arguments:
['/home/gperaza/anaconda3/envs/mlflow-7122f0cb71f385d249fbb61cc599afd8045ab238/bin/python', '-m', 'pip', 'install', '-U', '-r', 

 Tambien podemos ejecutar el proyecto directamente desde GitHub pasando la URL del repositorio público, y justo eso es lo que vamos a hacer en el challenge.

Una vez ejecutado, el run particular se agrega a la carpeta `mlruns`.